<p float="left">
  <img src="images/arrow_logo.jpg" alt="Arrow Logo" width="200" height="100" align="left"/>
  <img src="images/adi_logo.png" alt="ADI Logo" width="200" height="100" align="right"/>
</p>

<hr style="height:5px; text-align:bottom">

# Run The Cells Below
Run the cells below in sequence from top to bottom. Upon running the cell which initializes the required classes, a control widget and a pop-up containing data plots will appear.

To execute a cell:
1. Select the cell you wish to run by clicking the white space surrounding it. A blue border will appear around the cell.

    <img src="images/jupyter_cell_selected.PNG" alt="Jupyter Cell Selected"/>

2. Click the <strong>Run</strong> button at the top of the notebook to run the selected cell. Alternatively, you can use the CTRL+Enter keyboard combination to run the selected cell.
    - You can run the cells with markdown code in them, they have no effect on the program
    
3. After the cell has been run, a number indicating the current execution step will be inserted between the brackets on the left side of the cell.

    <img src="images/jupyter_cell_executed.PNG" alt="Jupyter Cell Executed"/>
    
    - Occasionally, after running the cell an asterisk may appear instead of a number, this indicates the cell is busy. Simply run the cell again to force/continue code execution.

    <img src="images/jupyter_cell_busy.PNG" alt="Jupyter Cell Busy"/>


If you make a mistake or run into problems, click <strong>Kernel -> Restart & Clear Output</strong> in the toolbar at the top of the page and start again. This clears the kernel of any variables or functions that may have been created in the previous program.

<hr style="height:1px">

## Safari 96Boards Demo

##### 1. First we need to import the Serial and Data modules required for the demo. We will also set the Matplotlib backend to provide a pop-up window for our data plots

In [ ]:
from safari_serial import Serial
from safari_data import Data

# Set the TkAgg backend for matplotlib
%matplotlib tk

#### 2. In order to share data between the Serial and Data modules, we will use a dictionary to store data and settings and then initialize the classes with this dictionary. The dictionary created below contains a nested dictionary for each enabled channel. For each enabled channel, we need to initialize some default settings such as plot title and color.

In [ ]:
# Create a dictionary containing all of the enabled channels
channels = {"0":{}, "4":{}, "6":{}}

#default fields for each channel
for channel in channels:
    # Number of elements to plot
    channels[channel]["max_elements"] = 300
    # Array for storing timestamps of when data was received
    channels[channel]["timestamps"]=[]
    # Array for storing channel voltages
    channels[channel]["voltages"]=[]
    # Array for storing the converted sensor values i.e. Temperature (deg. C) or Acceleration (G's)
    channels[channel]["values"]=[]
    # The axis object used for plotting. Each channel will get its own axis
    channels[channel]["axis"] = None
    # The annotation object used for displaying key values such as average, min, and max values for each channel
    channels[channel]["annotation"] = None
    # Boolean indicating if the annotation is visible. Default value is false since there are initially no values to annotate
    channels[channel]["annotation_visible"] = False
    # Text displayed by the annotation
    channels[channel]["annotation_text"] = ""
    # Line object used for the annotation
    channels[channel]["line"] = None
    # X-Y coordinates of the annotation box
    channels[channel]["annot_ax_xy"] = None
    # Color of the plot, default value is Black ('k')
    channels[channel]["axes_color"] = 'k'
    # Title of the plot
    channels[channel]["plot_title"] = f"Channel {channel}"
    # Y label of the plot when the voltages are displayed
    channels[channel]["y_label"] = ""
    # Y label of of the plot when values are displayed
    channels[channel]["y_label_converted"] = ""
    # Function for formatting voltage to plot, if None is specified, default operation is rounding voltage
    # to 4 decimal places: round(float(voltage), 4)
    channels[channel]["voltage_format_func"] = None
    # Function for converting voltage to a sensor value for plotting, if None is specified, default operation is 
    # rounding voltage to 4 decimal places: round(float(voltage), 4)
    channels[channel]["value_conversion_func"] = None

##### 3. The next thing we need to do is create functions to format the voltages for plotting. For example, the channel 4 voltage will be represented in millivolts so we will multiply the received voltage by 1000. The voltages for channels 0 and 6 will simply be rounded to 4 decimal places.

In [ ]:
# Since we don't need to modify the default operation for formatting voltage on Channel 0 or Channel 6 (rounding to 4 decimals),
# we only need to create a format function for Channel 4 since it will be displayed in millivolts rather than volts

# Function for formatting channel 4 voltage for plotting
def convert_channel_4_voltage(voltage):
    return round(float(voltage) * 1000, 4)

#### 4. In addition to functions for formatting voltages, we need to create functions to convert the voltages from the sensors into actual units.

In [ ]:
# Function for converting channel 0 voltage to an accelerometer value
def convert_accelerometer(voltage, code):
    del code
    
    vcc = 3.3
    return round((voltage - vcc / 2) * 1 / 0.640, 3)

# Function for converting channel 4 code to a temperature value
def convert_temperature(voltage, code):
    del voltage
    
    r_rtd = (code * 5110) / ((2**24) * 16)
    temp = (r_rtd - 100) / 0.385
    return round(temp, 2)

# Function for converting channel 6 voltage to a pressure value
def convert_pressure(voltage, code):
    del code
    
    vcc = 3.3
    r1 = 132000
    r2 = 100000
    return round(((190.0 * voltage * (r1/r2)) / vcc - 38), 3)

#### 5. Now we need to customize each channel with our desired settings. This includes things such as plot and axis titles as well as setting the conversion functions we defined above.

In [ ]:
# Configure Channel 0
channels["0"]["axes_color"] = 'tab:blue'
channels["0"]["plot_title"] = "Channel 0"
channels["0"]["y_label"] = 'Voltage (V)'
channels["0"]["y_label_converted"] = 'Accel. (g)'
channels["0"]["value_conversion_func"] = convert_accelerometer

# Configure Channel 4
channels["4"]["axes_color"] = 'tab:red'
channels["4"]["plot_title"] = "Channel 4"
channels["4"]["y_label"] = 'Voltage (mV)'
channels["4"]["y_label_converted"] = 'Temp. (deg C)'
channels["4"]["voltage_format_func"] = convert_channel_4_voltage
channels["4"]["value_conversion_func"] = convert_temperature

# Configure Channel 6
channels["6"]["axes_color"] = 'tab:green'
channels["6"]["plot_title"] = "Channel 6"
channels["6"]["y_label"] = 'Voltage (V)'
channels["6"]["y_label_converted"] = 'Pressure (Pa)'
channels["6"]["value_conversion_func"] = convert_pressure

#### 6. Finally we simply initialize the Serial and Data class objects with the 'channels' dictionary created previously along with the desired configuration file. When the Serial class object is initialized, button widgets will be created allowing you to connect to the serial port of the Meerkat as well as start and stop data collection from the Safari 96Board. When the Data class object is initialized, a pop-up window will be generated displaying the plots for enabled channel and a radio button widget will be created allowing you to switch the plots from displaying voltage to the converted sensor value.

###### These classes can be viewed and edited using the supplied safari_serial.py and safari_data.py files

<span style='color:red'><strong>Please note that the configuration file will be written to the board once the connect button is clicked.</strong></span>

In [ ]:
#Initialize the classes with the dictionary containing the channel information
serial = Serial(channels, 'ad7124_default_config.py')
data = Data(channels)